# Parameter optimization

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'buy'
pattern = ['Pattern', 'Trend']
indicator_list = pattern
indicator_list_higher = pattern

work_timeframe = '15m'
higher_timeframe = '1h'
opt_limit = 2000
load = False

print(f'Timeframe is {higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Indicator_list'] = pattern
configs['Higher_TF_indicator_list'] = indicator_list_higher
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'Pattern': {
                            'use_vol': [1], 
                            'window_low_bound': [1], 
                            'window_high_bound': [4],
                            'first_candle': [0.8],
                            'second_candle':[0.5],
                            'third_candle': [0.5]
                            },
                'Trend': {
                            'timeperiod': np.arange(2, 102, 2),
                            'low_bound': [0]
                            }
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

# stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
# if not stat_list:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
# else:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')


Timeframe is 1h, trade type is buy
Number of combinations is 50


 62%|█████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                              | 31/50 [03:26<01:33,  4.90s/it]

# Check local statistics

In [2]:
# candles
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,Pattern_use_vol,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Pattern_third_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
0,Pattern_Trend,0,1,4,0.8,0.5,0.5,28,0,1034,2.109483,1.065,1147.205767,1101.21,1.4216,1.4202,1.4162,1.5521,1.5919,1.64,1.7457,1.8163,1.8364,1.8989,1.9205,1.9527,2.302,2.3459,2.3951,2.4617,2.5089,2.5702,2.5891,2.6049,2.6211,2.6439,2.6571,2.7152,0.08,0.16,0.23,0.31,0.39,0.48,0.57,0.65,0.74,0.82,0.91,1.0,1.09,1.18,1.27,1.38,1.48,1.58,1.67,1.75,1.84,1.9,1.99,2.09


In [9]:
# candles
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,Pattern_use_vol,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Pattern_third_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
49,Pattern_Trend,0,1,5,0.70,0.95,0.5,28,0,406,2.079163,1.058750,438.139975,429.852500,1.5254,1.5608,1.6136,1.7397,1.7779,1.7780,1.8998,1.9796,1.9756,2.0476,1.9924,2.0108,2.0565,2.0605,2.1289,2.2079,2.2600,2.3752,2.4224,2.4732,2.4811,2.4976,2.5154,2.5200,0.07,0.16,0.24,0.33,0.41,0.49,0.58,0.66,0.73,0.82,0.90,0.98,1.07,1.15,1.25,1.34,1.44,1.53,1.63,1.73,1.83,1.93,2.02,2.12
59,Pattern_Trend,0,1,5,0.75,0.95,0.5,28,0,350,2.058383,1.055000,370.434167,369.250000,1.5911,1.6073,1.6455,1.7232,1.7590,1.7367,1.8686,1.9511,1.9283,2.0072,1.9487,1.9818,2.0217,2.0224,2.0902,2.1519,2.2129,2.3545,2.3882,2.4425,2.4594,2.4815,2.5042,2.5233,0.08,0.17,0.26,0.34,0.42,0.51,0.59,0.67,0.73,0.81,0.90,0.98,1.06,1.14,1.23,1.33,1.42,1.51,1.61,1.71,1.81,1.91,2.01,2.12
69,Pattern_Trend,0,1,5,0.80,0.95,0.5,28,0,294,2.163387,1.052917,342.035925,309.557500,1.5921,1.6627,1.6530,1.7869,1.8045,1.7776,1.9317,2.0187,2.0220,2.0997,2.0800,2.1061,2.1353,2.1421,2.2251,2.2964,2.3474,2.4875,2.5412,2.6168,2.6247,2.6383,2.6605,2.6710,0.07,0.15,0.23,0.31,0.39,0.47,0.56,0.64,0.70,0.79,0.88,0.96,1.05,1.13,1.23,1.33,1.43,1.53,1.63,1.74,1.85,1.96,2.06,2.18
99,Pattern_Trend,0,1,5,0.95,0.95,0.5,28,0,181,2.303404,1.039167,235.916154,188.089167,1.6804,1.6609,1.6833,1.8144,1.8878,1.9644,2.0948,2.1690,2.1926,2.2462,2.2857,2.3174,2.2788,2.2842,2.3746,2.4299,2.4800,2.6145,2.6884,2.8019,2.7803,2.8136,2.8804,2.8582,0.07,0.15,0.22,0.30,0.38,0.47,0.55,0.63,0.70,0.78,0.87,0.95,1.03,1.11,1.21,1.31,1.40,1.51,1.61,1.72,1.83,1.94,2.04,2.16
79,Pattern_Trend,0,1,5,0.85,0.95,0.5,28,0,260,2.276008,1.032083,331.762167,268.341667,1.6004,1.6642,1.6897,1.8219,1.8455,1.8656,2.0323,2.1310,2.1378,2.2118,2.1974,2.2256,2.2518,2.2439,2.3366,2.4155,2.4683,2.6320,2.6915,2.7775,2.7828,2.8289,2.8798,2.8924,0.07,0.14,0.21,0.29,0.36,0.44,0.52,0.60,0.66,0.74,0.83,0.92,1.01,1.10,1.20,1.31,1.41,1.52,1.62,1.73,1.85,1.96,2.08,2.20
70,Pattern_Trend,0,1,5,0.85,0.50,0.5,28,0,1078,2.137050,1.028750,1225.739900,1108.992500,1.4064,1.4222,1.4692,1.5796,1.6164,1.6647,1.7675,1.8447,1.8789,1.9044,1.9382,1.9726,2.3036,2.3703,2.4218,2.4891,2.5418,2.6156,2.6188,2.6476,2.6642,2.6967,2.7157,2.7392,0.08,0.15,0.23,0.32,0.39,0.48,0.56,0.64,0.72,0.80,0.88,0.97,1.05,1.14,1.23,1.33,1.43,1.52,1.60,1.68,1.76,1.82,1.91,2.00
39,Pattern_Trend,0,1,5,0.65,0.95,0.5,28,0,465,2.158138,1.025417,538.533937,476.818750,1.5742,1.5842,1.6485,1.7458,1.8104,1.8356,1.9849,2.0524,2.0427,2.1303,2.0705,2.0974,2.1562,2.1562,2.2206,2.2976,2.3534,2.4551,2.4977,2.5897,2.5970,2.6215,2.6375,2.6359,0.07,0.15,0.23,0.31,0.39,0.47,0.55,0.63,0.70,0.78,0.86,0.94,1.02,1.11,1.20,1.30,1.39,1.49,1.59,1.69,1.79,1.89,1.98,2.08
67,Pattern_Trend,0,1,5,0.80,0.85,0.5,28,0,524,2.056721,1.025000,553.721717,537.100000,1.4602,1.5317,1.5650,1.7116,1.7436,1.7460,1.8569,1.9397,1.9379,2.0024,1.9917,2.0153,2.0144,2.0499,2.1475,2.2185,2.2471,2.3488,2.3878,2.4453,2.4674,2.5175,2.5070,2.5081,0.07,0.15,0.23,0.32,0.40,0.49,0.57,0.65,0.72,0.80,0.88,0.96,1.04,1.12,1.21,1.30,1.40,1.49,1.57,1.66,1.75,1.85,1.94,2.03
77,Pattern_Trend,0,1,5,0.85,0.85,0.5,28,0,448,2.070333,1.021250,479.509333,457.520000,1.4664,1.521

In [ ]:
# LR timeperiod one candle
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

In [13]:
# LR timeperiod
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,Pattern_use_vol,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Pattern_third_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
26,Pattern_Trend,0,1,5,0.8,0.5,0.5,54,0,1424,2.032267,1.029167,1469.947733,1465.533333,1.3690,1.4023,1.4215,1.5193,1.5738,1.6357,1.7326,1.8127,1.8468,1.8583,1.8852,1.9198,2.1791,2.2263,2.2767,2.3405,2.3870,2.4434,2.4506,2.4606,2.4812,2.4988,2.5124,2.5408,0.08,0.15,0.23,0.31,0.39,0.48,0.56,0.65,0.73,0.81,0.89,0.98,1.06,1.15,1.24,1.34,1.43,1.52,1.60,1.68,1.75,1.81,1.89,1.97
27,Pattern_Trend,0,1,5,0.8,0.5,0.5,56,0,1428,2.026333,1.026667,1465.604000,1466.080000,1.3728,1.4003,1.4211,1.5181,1.5724,1.6306,1.7254,1.8055,1.8399,1.8524,1.8804,1.9156,2.1718,2.2193,2.2681,2.3321,2.3790,2.4356,2.4432,2.4527,2.4733,2.4911,2.5015,2.5298,0.08,0.15,0.23,0.31,0.39,0.48,0.56,0.65,0.73,0.81,0.89,0.97,1.06,1.15,1.24,1.33,1.43,1.52,1.60,1.67,1.75,1.80,1.88,1.96
24,Pattern_Trend,0,1,5,0.8,0.5,0.5,50,0,1407,2.036208,1.026250,1457.945125,1443.933750,1.3766,1.4014,1.4230,1.5247,1.5766,1.6373,1.7339,1.8142,1.8469,1.8600,1.8866,1.9193,2.1819,2.2295,2.2821,2.3452,2.3922,2.4504,2.4574,2.4674,2.4879,2.5065,2.5202,2.5478,0.08,0.15,0.23,0.31,0.39,0.48,0.56,0.65,0.73,0.81,0.89,0.97,1.06,1.15,1.23,1.33,1.43,1.52,1.60,1.67,1.75,1.80,1.88,1.96
30,Pattern_Trend,0,1,5,0.8,0.5,0.5,62,0,1459,2.026417,1.026250,1497.541917,1497.298750,1.3886,1.4120,1.4355,1.5336,1.5857,1.6468,1.7273,1.8080,1.8431,1.8593,1.8846,1.9180,2.1698,2.2169,2.2641,2.3260,2.3731,2.4235,2.4303,2.4398,2.4605,2.4786,2.4916,2.5173,0.08,0.16,0.23,0.32,0.40,0.48,0.57,0.65,0.73,0.81,0.89,0.98,1.06,1.15,1.24,1.33,1.42,1.51,1.59,1.66,1.74,1.80,1.87,1.96
21,Pattern_Trend,0,1,5,0.8,0.5,0.5,44,0,1377,2.040037,1.025000,1432.131637,1411.425000,1.3623,1.3968,1.4129,1.5165,1.5697,1.6317,1.7297,1.8090,1.8477,1.8704,1.8961,1.9277,2.1939,2.2419,2.2959,2.3588,2.4028,2.4602,2.4651,2.4757,2.4954,2.5158,2.5287,2.5562,0.08,0.15,0.23,0.31,0.39,0.47,0.56,0.64,0.72,0.80,0.89,0.97,1.06,1.14,1.23,1.33,1.43,1.52,1.60,1.67,1.75,1.81,1.88,1.97
28,Pattern_Trend,0,1,5,0.8,0.5,0.5,58,0,1437,2.026217,1.024583,1474.673350,1472.326250,1.3754,1.4024,1.4274,1.5265,1.5794,1.6422,1.7214,1.8020,1.8369,1.8525,1.8794,1.9140,2.1715,2.2194,2.2680,2.3317,2.3794,2.4307,2.4381,2.4475,2.4684,2.4866,2.5015,2.5269,0.08,0.15,0.23,0.31,0.39,0.48,0.56,0.64,0.72,0.80,0.89,0.97,1.06,1.15,1.23,1.33,1.43,1.51,1.60,1.67,1.75,1.80,1.88,1.96
31,Pattern_Trend,0,1,5,0.8,0.5,0.5,64,0,1472,2.025071,1.021250,1508.904267,1503.280000,1.3833,1.4065,1.4334,1.5314,1.5868,1.6492,1.7309,1.8120,1.8447,1.8614,1.8822,1.9194,2.1701,2.2176,2.2648,2.3226,2.3689,2.4195,2.4248,2.4342,2.4564,2.4747,2.4907,2.5162,0.08,0.15,0.23,0.31,0.39,0.48,0.56,0.64,0.72,0.81,0.89,0.97,1.06,1.14,1.23,1.32,1.42,1.51,1.59,1.66,1.74,1.79,1.87,1.95
29,Pattern_Trend,0,1,5,0.8,0.5,0.5,60,0,1446,2.022158,1.020417,1478.040950,1475.522500,1.3777,1.4049,1.4285,1.5275,1.5800,1.6421,1.7220,1.8009,1.8357,1.8512,1.8772,1.9110,2.1643,2.2123,2.2604,2.3238,2.3716,2.4228,2.4304,2.4399,2.4607,2.4785,2.4916,2.5168,0.08,0.15,0.23,0.31,0.39,0.48,0.56,0.64,0.72,0.80,0.89,0.97,1.05,1.14,1.23,1.32,1.42,1.51,1.59,1.66,1.74,1.79,1.87,1.95
20,Pattern_Trend,0,1,5,0.8,0.5,0.5,42,0,1367,2.037842,1.020417,1418.729558,1394.909583,1.3587

In [2]:
# vol
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,Pattern_use_vol,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Pattern_third_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
2,Pattern_Trend,1,1,5,0.8,0.5,0.5,28,0,117,2.721938,1.287083,201.466688,150.588750,1.6343,1.8121,1.8077,2.1569,2.2423,2.2469,2.2988,2.3666,2.3793,2.4783,2.5767,2.7266,2.7558,2.8295,2.9710,3.0502,3.0906,3.2132,3.2524,3.3021,3.2832,3.3928,3.6759,3.7833,0.07,0.15,0.23,0.32,0.41,0.51,0.61,0.70,0.78,0.90,1.04,1.16,1.29,1.42,1.56,1.71,1.86,1.99,2.14,2.27,2.41,2.29,2.46,2.61
0,Pattern_Trend,-1,1,5,0.8,0.5,0.5,28,0,973,1.991688,1.038333,964.911938,1010.298333,1.3548,1.3729,1.3539,1.4297,1.4860,1.5418,1.6474,1.7221,1.7658,1.7952,1.8123,1.8344,2.1827,2.2239,2.2573,2.3209,2.3618,2.4302,2.4403,2.4636,2.4844,2.4983,2.4990,2.5218,0.08,0.16,0.24,0.33,0.41,0.50,0.58,0.67,0.75,0.83,0.92,1.00,1.07,1.16,1.25,1.34,1.43,1.52,1.59,1.67,1.74,1.82,1.89,1.97
1,Pattern_Trend,0,1,5,0.8,0.5,0.5,28,0,1267,2.022196,1.019583,1295.122121,1291.812083,1.3566,1.3828,1.3841,1.4915,1.5454,1.5976,1.6958,1.7785,1.8123,1.8405,1.8598,1.8944,2.1758,2.2282,2.2807,2.3418,2.3830,2.4511,2.4514,2.4734,2.4922,2.5141,2.5347,2.5670,0.08,0.15,0.23,0.31,0.39,0.47,0.56,0.64,0.72,0.80,0.88,0.96,1.04,1.13,1.22,1.32,1.42,1.51,1.59,1.66,1.74,1.80,1.88,1.97


In [10]:
# vol one candle
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,Pattern_use_vol,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Pattern_third_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
2,Pattern_Trend,1,1,4,0.8,0.5,0.5,72,0,110,2.711833,1.280417,188.301667,140.845833,1.9388,1.8673,1.9106,2.2016,2.2753,2.1896,2.2118,2.2852,2.3252,2.4708,2.5940,2.7991,2.8507,2.8088,2.8987,2.9701,3.0039,3.1017,3.2357,3.2571,3.2636,3.2985,3.5409,3.7850,0.08,0.15,0.24,0.33,0.42,0.51,0.60,0.69,0.80,0.92,1.04,1.17,1.30,1.42,1.55,1.69,1.83,1.97,2.12,2.25,2.38,2.26,2.42,2.59
0,Pattern_Trend,-1,1,4,0.8,0.5,0.5,72,0,902,2.062754,1.111250,958.604258,1002.347500,1.3925,1.4188,1.3958,1.5093,1.5842,1.6457,1.7290,1.7784,1.8121,1.8686,1.8833,1.9249,2.3181,2.3319,2.3468,2.3941,2.4496,2.4932,2.5128,2.5251,2.5427,2.5570,2.5317,2.5605,0.09,0.17,0.25,0.34,0.43,0.53,0.62,0.71,0.80,0.89,0.98,1.07,1.16,1.25,1.34,1.44,1.53,1.63,1.71,1.78,1.86,1.94,2.03,2.12
1,Pattern_Trend,0,1,4,0.8,0.5,0.5,72,0,2742,1.941346,0.942083,2581.170275,2583.192500,1.2645,1.3557,1.3830,1.4524,1.6663,1.6909,1.7445,1.7864,1.8051,1.8411,1.8648,1.9089,2.0679,2.0981,2.1077,2.1481,2.1916,2.2302,2.2518,2.2690,2.2848,2.3101,2.4188,2.4506,0.08,0.15,0.23,0.30,0.38,0.46,0.54,0.61,0.69,0.76,0.83,0.90,0.98,1.05,1.13,1.21,1.30,1.37,1.45,1.51,1.58,1.63,1.70,1.77


In [2]:
# high/low bound
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,Pattern_use_vol,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Pattern_third_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
0,Pattern_Trend,0,1,3,0.8,0.5,0.5,28,0,516,2.494167,1.195417,770.990000,616.835000,1.3335,1.3897,1.4774,1.7375,1.7577,1.8068,1.9048,2.0067,2.0177,2.1010,2.1601,2.2063,2.9277,2.9553,3.0383,3.0905,3.1330,3.2269,3.1853,3.1666,3.2086,3.2565,3.3369,3.4352,0.08,0.16,0.25,0.34,0.44,0.54,0.64,0.73,0.83,0.92,1.02,1.11,1.21,1.32,1.43,1.54,1.66,1.78,1.88,1.96,2.06,2.15,2.26,2.38
3,Pattern_Trend,0,1,6,0.8,0.5,0.5,28,0,1436,1.994946,1.077917,1428.742217,1547.888333,1.3832,1.4119,1.4300,1.5334,1.5768,1.6380,1.7050,1.7638,1.7907,1.8257,1.8385,1.8835,2.1362,2.1754,2.2175,2.2786,2.3234,2.3840,2.3823,2.3915,2.4082,2.4252,2.4518,2.5241,0.08,0.16,0.24,0.33,0.42,0.51,0.60,0.69,0.77,0.86,0.94,1.03,1.12,1.21,1.31,1.40,1.50,1.59,1.67,1.74,1.82,1.88,1.96,2.04
1,Pattern_Trend,0,1,4,0.8,0.5,0.5,28,0,1030,2.131912,1.071667,1165.869875,1103.816667,1.4365,1.4379,1.4291,1.5583,1.6010,1.6500,1.7593,1.8334,1.8571,1.9217,1.9419,1.9735,2.3295,2.3745,2.4238,2.4916,2.5384,2.6010,2.6205,2.6341,2.6501,2.6728,2.6847,2.7452,0.08,0.16,0.23,0.31,0.40,0.49,0.57,0.66,0.75,0.83,0.92,1.01,1.10,1.19,1.28,1.39,1.49,1.59,1.68,1.76,1.85,1.90,1.99,2.09
2,Pattern_Trend,0,1,5,0.8,0.5,0.5,28,0,1241,2.031775,1.045833,1280.432775,1297.879167,1.3912,1.3891,1.3970,1.5208,1.5659,1.6086,1.7054,1.7677,1.8006,1.8379,1.8503,1.8886,2.1859,2.2353,2.2847,2.3513,2.3982,2.4641,2.4670,2.4832,2.4996,2.5180,2.5393,2.6129,0.08,0.15,0.23,0.31,0.39,0.48,0.57,0.65,0.74,0.82,0.90,0.99,1.07,1.16,1.26,1.36,1.46,1.55,1.63,1.71,1.79,1.85,1.93,2.02
4,Pattern_Trend,0,1,7,0.8,0.5,0.5,28,0,1610,1.930638,1.037917,1498.326375,1671.045833,1.2938,1.3308,1.3806,1.4722,1.5205,1.5946,1.6520,1.7090,1.7487,1.7853,1.8011,1.8427,2.0727,2.1081,2.1386,2.2046,2.2431,2.3012,2.3028,2.3162,2.3337,2.3532,2.3799,2.4499,0.08,0.15,0.23,0.32,0.40,0.49,0.58,0.67,0.75,0.83,0.91,1.00,1.08,1.17,1.26,1.35,1.44,1.53,1.60,1.67,1.75,1.81,1.88,1.96


In [2]:
# high/low bound
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,Pattern_use_vol,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Pattern_third_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
0,Pattern_Trend,0,1,3,0.8,0.5,0.5,54,0,579,2.441417,1.175000,834.580250,680.325000,1.3040,1.3708,1.4595,1.6787,1.7116,1.7917,1.9049,2.0053,2.0219,2.0938,2.1539,2.2100,2.8669,2.8899,2.9616,3.0032,3.0526,3.1279,3.0977,3.0762,3.1143,3.1560,3.2280,3.3136,0.08,0.16,0.24,0.34,0.43,0.53,0.62,0.72,0.81,0.91,1.00,1.10,1.21,1.31,1.42,1.53,1.64,1.75,1.85,1.93,2.02,2.11,2.19,2.30
3,Pattern_Trend,0,1,6,0.8,0.5,0.5,54,0,1622,1.975525,1.064167,1582.301550,1726.078333,1.3982,1.4213,1.4537,1.5448,1.5879,1.6428,1.7052,1.7648,1.7929,1.8133,1.8344,1.8755,2.1066,2.1428,2.1788,2.2392,2.2897,2.3409,2.3470,2.3487,2.3675,2.3841,2.4002,2.4323,0.08,0.16,0.24,0.33,0.41,0.50,0.59,0.68,0.77,0.85,0.93,1.02,1.11,1.20,1.29,1.39,1.48,1.57,1.65,1.72,1.79,1.85,1.93,2.00
1,Pattern_Trend,0,1,4,0.8,0.5,0.5,54,0,1146,2.119783,1.060000,1283.271700,1214.760000,1.4245,1.4352,1.4501,1.5753,1.6152,1.6726,1.7836,1.8552,1.8720,1.9121,1.9389,1.9756,2.3003,2.3429,2.3911,2.4588,2.5163,2.5655,2.5909,2.6002,2.6196,2.6367,2.6468,2.6954,0.08,0.15,0.23,0.31,0.39,0.48,0.57,0.65,0.74,0.82,0.91,1.00,1.09,1.18,1.28,1.38,1.48,1.58,1.67,1.74,1.83,1.88,1.96,2.04
2,Pattern_Trend,0,1,5,0.8,0.5,0.5,54,0,1398,2.014721,1.037917,1418.579725,1451.007500,1.3912,1.3956,1.4186,1.5323,1.5766,1.6268,1.7225,1.7826,1.8152,1.8302,1.8501,1.8883,2.1581,2.2035,2.2488,2.3131,2.3676,2.4226,2.4323,2.4374,2.4561,2.4741,2.4877,2.5220,0.08,0.15,0.23,0.31,0.39,0.48,0.56,0.65,0.73,0.81,0.90,0.98,1.07,1.16,1.25,1.35,1.45,1.54,1.62,1.70,1.77,1.83,1.91,1.99
4,Pattern_Trend,0,1,7,0.8,0.5,0.5,54,0,1823,1.917338,1.037083,1672.306263,1890.602917,1.3224,1.3601,1.4166,1.4957,1.5409,1.6056,1.6645,1.7228,1.7616,1.7815,1.7983,1.8358,2.0444,2.0776,2.1019,2.1666,2.2097,2.2599,2.2688,2.2758,2.2958,2.3147,2.3305,2.3646,0.08,0.16,0.24,0.32,0.40,0.49,0.58,0.67,0.75,0.83,0.92,1.00,1.09,1.17,1.26,1.35,1.44,1.52,1.60,1.67,1.74,1.80,1.87,1.94


# Research parameter influence

In [3]:
param = 'Pattern_use_vol'

res = stat.groupby([param]).agg({'e_ratio_avg': 'mean',
                                 'pct_price_diff_avg': 'mean',
                                 'e_ratio_rank': 'mean', 
                                 'price_rank': 'mean', 
                                 'forecasts_num': 'sum'}).sort_values(param).reset_index()
res

,Pattern_use_vol,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,forecasts_num
0,-1,1.546973,2.873992,1061.440504,5577.185392,291081
1,0,1.387825,2.958303,865.087474,6598.829933,334592
2,1,1.387787,2.958333,865.153912,6600.041667,334650


# Plot result in file

In [23]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

x, y = param, 'e_ratio_avg'
ax.plot(res[x], res[y])
ax.set_xlabel(x)
ax.set_ylabel(y)
ax.set_title('_'.join(pattern) + '_' + ttype)

fig.savefig(f"{'_'.join(pattern)}'_'{ttype}_{param}", dpi=fig.dpi)

# Save new config data to config file

In [3]:
from config_updater import ConfigUpdater

ttype = 'buy'
pattern = ['Pattern', 'Trend']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                 'Pattern': {
                            'use_vol': [0], 
                            'window_low_bound': [1], 
                            'window_high_bound': [4],
                            'first_candle': [0.8],
                            'second_candle': [0.5],
                            'third_candle': [0.5]
                            },
                'Trend': {
                            'timeperiod': [28], 
                            'low_bound': [0]
                            }
             }
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)